In [1]:
import snowflake.connector

# Set up connection parameters
ACCOUNT = 'yw24586.uk-south.azure'
USER = 'TOMMYMCD22'
PASSWORD = 'Snowflake1!'
WAREHOUSE = 'COMPUTE_WH'
DATABASE = 'EXAM'
SCHEMA = 'DBT_TMCDONAGH'

# Create a Snowflake connection object
conn = snowflake.connector.connect(
    account=ACCOUNT,
    user=USER,
    password=PASSWORD,
    warehouse=WAREHOUSE,
    database=DATABASE,
    schema=SCHEMA
)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/snowflake/connector/options.py:108: UserWarning: You have an incompatible version of 'pyarrow' installed (11.0.0), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
# Create a cursor object
cursor_1 = conn.cursor()
cursor_2 = conn.cursor()

# Execute a SELECT statement to extract data from a Snowflake table
encoded = cursor_1.execute('SELECT * FROM EXAM_ENCODED')
plain = cursor_2.execute('SELECT * FROM EXAM_PLAIN')

# Fetch the results into a pandas DataFrame
import pandas as pd
encoded_results = encoded.fetchall()
encoded_df = pd.DataFrame(encoded_results, columns=[desc[0] for desc in cursor_1.description])

plain_results = plain.fetchall()
plain_df = pd.DataFrame(plain_results, columns=[desc[0] for desc in cursor_2.description])


In [28]:
plain_df

,GENDER,RACE,PARENT_EDUCATION,LUNCH,TEST_PREP,MATH_SCORE,READING_SCORE,WRITING_SCORE
0,female,group_B,bachelors_degree,standard,none,72,72,74
1,female,group_C,some_college,standard,completed,69,90,88
2,female,group_B,masters_degree,standard,none,90,95,93
3,male,group_A,associates_degree,free_reduced,none,47,57,44
4,male,group_C,some_college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group_E,masters_degree,standard,completed,88,99,95
996,male,group_C,high_school,free_reduced,none,62,55,55
997,female,group_C,high_school,free_reduced,completed,59,71,65
998,female,group_D,some_college,standard,completed,68,78,77


In [27]:
encoded_df

,GENDER_MALE,GENDER_FEMALE,RACE_GROUP_A,RACE_GROUP_B,RACE_GROUP_C,RACE_GROUP_D,RACE_GROUP_E,PARENT_EDUCATION_SOME_COLLEGE,PARENT_EDUCATION_ASSOCIATES_DEGREE,PARENT_EDUCATION_HIGH_SCHOOL,PARENT_EDUCATION_SOME_HIGH_SCHOOL,PARENT_EDUCATION_BACHELORS_DEGREE,PARENT_EDUCATION_MASTERS_DEGREE,LUNCH_STANDARD,LUNCH_FREE_REDUCED,TEST_PREP_NONE,TEST_PREP_COMPLETED,MATH_SCORE,READING_SCORE,WRITING_SCORE
0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0,72,72,74
1,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,69,90,88
2,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,90,95,93
3,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,47,57,44
4,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,76,78,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,88,99,95
996,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,62,55,55
997,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,59,71,65
998,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,68,78,77


In [88]:
# correlation function
import seaborn as sns
import matplotlib.pyplot as plt

def plot_correlation_matrix(df, target_col):
    corr = df.corr()[target_col].abs().sort_values(by=target_col, ascending=False)
    return corr


In [96]:
math_corr = plot_correlation_matrix(encoded_df.drop(columns=['READING_SCORE', 'WRITING_SCORE']), ['MATH_SCORE'])
math_corr.rename(columns={'MATH_SCORE':'MATH_SCORE Correlations'}, inplace=True)
math_corr_10 = math_corr[1:11]
math_corr_10

,MATH_SCORE Correlations
LUNCH_FREE_REDUCED,0.350877
LUNCH_STANDARD,0.350877
RACE_GROUP_E,0.205855
TEST_PREP_COMPLETED,0.177702
TEST_PREP_NONE,0.177702
GENDER_FEMALE,0.167982
GENDER_MALE,0.167982
PARENT_EDUCATION_HIGH_SCHOOL,0.128725
RACE_GROUP_A,0.091977
RACE_GROUP_B,0.084250


In [97]:
reading_corr = plot_correlation_matrix(encoded_df.drop(columns=['MATH_SCORE', 'WRITING_SCORE']), ['READING_SCORE'])
reading_corr.rename(columns={'READING_SCORE':'READING_SCORE Correlations'}, inplace=True)
reading_corr_10 = reading_corr[1:11]
reading_corr_10

,READING_SCORE Correlations
GENDER_MALE,0.244313
GENDER_FEMALE,0.244313
TEST_PREP_COMPLETED,0.241780
TEST_PREP_NONE,0.241780
LUNCH_STANDARD,0.229560
LUNCH_FREE_REDUCED,0.229560
PARENT_EDUCATION_HIGH_SCHOOL,0.151068
RACE_GROUP_E,0.106712
PARENT_EDUCATION_MASTERS_DEGREE,0.106452
RACE_GROUP_A,0.096274


In [98]:
writing_corr = plot_correlation_matrix(encoded_df.drop(columns=['READING_SCORE', 'MATH_SCORE']), ['WRITING_SCORE'])
writing_corr.rename(columns={'WRITING_SCORE':'WRITING_SCORE Correlations'}, inplace=True)
writing_corr_10 = writing_corr[1:11]
writing_corr_10

,WRITING_SCORE Correlations
TEST_PREP_NONE,0.312946
TEST_PREP_COMPLETED,0.312946
GENDER_MALE,0.301225
GENDER_FEMALE,0.301225
LUNCH_STANDARD,0.245769
LUNCH_FREE_REDUCED,0.245769
PARENT_EDUCATION_HIGH_SCHOOL,0.182211
PARENT_EDUCATION_BACHELORS_DEGREE,0.128297
PARENT_EDUCATION_MASTERS_DEGREE,0.125693
RACE_GROUP_A,0.110714


In [104]:
math_corr_10.index.values

array(['LUNCH_FREE_REDUCED', 'LUNCH_STANDARD', 'RACE_GROUP_E',
       'TEST_PREP_COMPLETED', 'TEST_PREP_NONE', 'GENDER_FEMALE',
       'GENDER_MALE', 'PARENT_EDUCATION_HIGH_SCHOOL', 'RACE_GROUP_A',
       'RACE_GROUP_B'], dtype=object)

# Math score predictions

In [109]:
from sklearn.model_selection import train_test_split
from numpy import random
rng = random.RandomState(123)

# X = encoded_df.drop(columns=['MATH_SCORE', 'READING_SCORE', 'WRITING_SCORE'])
X = encoded_df[math_corr_10.index]
y_math = plain_df['MATH_SCORE']

X_train, X_test, y_train_math, y_test_math = train_test_split(X, y_math, test_size=0.2, random_state=rng)

print('Encoded data train-test split:')
print(f'{X_train.shape=}, {X_test.shape=}, {y_train_math.shape=}, {y_test_math.shape=}')


Encoded data train-test split:
X_train.shape=(800, 10), X_test.shape=(200, 10), y_train_math.shape=(800,), y_test_math.shape=(200,)


In [110]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

forest = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=rng)
gradient_boost = XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.2, random_state=rng)


forest = forest.fit(X_train, y_train_math)
forest_math = forest.predict(X_test)

boost = gradient_boost.fit(X_train, y_train_math)
boost_math = boost.predict(X_test)

In [111]:
# helper function to evaluate the models
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def regression_metrics(y_test, y_pred):
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    return {'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'R2': r2}


In [124]:
math_forest_metrics = regression_metrics(y_test_math, forest_math)
math_boost_metrics = regression_metrics(y_test_math, boost_math)

print(f'Math score random forest regression metrics:\n{math_forest_metrics}')
print(f'\nMath score XGBoost metrics:\n{math_boost_metrics}')

Math score random forest regression metrics:
{'MAE': 11.293742447463071, 'MSE': 189.7850047370336, 'RMSE': 13.776247846820759, 'R2': 0.18263621622387027}

Math score XGBoost metrics:
{'MAE': 11.288140926361084, 'MSE': 192.61356790970873, 'RMSE': 13.878529025430208, 'R2': 0.17045419425289832}


# Reading score predictions

In [114]:
X = encoded_df[reading_corr_10.index]
y_reading = plain_df['READING_SCORE']

X_train, X_test, y_train_reading, y_test_reading = train_test_split(X, y_reading, test_size=0.2, random_state=rng)

print('Encoded data train-test split:')
print(f'{X_train.shape=}, {X_test.shape=}, {y_train_reading.shape=}, {y_test_reading.shape=}')

Encoded data train-test split:
X_train.shape=(800, 10), X_test.shape=(200, 10), y_train_reading.shape=(800,), y_test_reading.shape=(200,)


In [115]:
forest = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=rng)
gradient_boost = XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.2, random_state=rng)


forest = forest.fit(X_train, y_train_reading)
forest_reading = forest.predict(X_test)

boost = gradient_boost.fit(X_train, y_train_reading)
boost_reading = boost.predict(X_test)

In [125]:
reading_forest_metrics = regression_metrics(y_test_reading, forest_reading)
reading_boost_metrics = regression_metrics(y_test_reading, boost_reading)

print(f'Reading score random forest regression metrics:\n{reading_forest_metrics}')
print(f'\nReading score XGBoost metrics:\n{reading_boost_metrics}')

Reading score random forest regression metrics:
{'MAE': 11.404424014000247, 'MSE': 195.10418700102258, 'RMSE': 13.967970038664264, 'R2': 0.035575253911013904}

Reading score XGBoost metrics:
{'MAE': 11.745195484161377, 'MSE': 208.05755126420766, 'RMSE': 14.42420019495735, 'R2': -0.02845486882774062}


# Writing score predictions

In [117]:
X = encoded_df[writing_corr_10.index]
y_writing = plain_df['READING_SCORE']

X_train, X_test, y_train_writing, y_test_writing = train_test_split(X, y_writing, test_size=0.2, random_state=rng)

print('Encoded data train-test split:')
print(f'{X_train.shape=}, {X_test.shape=}, {y_train_writing.shape=}, {y_test_writing.shape=}')

Encoded data train-test split:
X_train.shape=(800, 10), X_test.shape=(200, 10), y_train_writing.shape=(800,), y_test_writing.shape=(200,)


In [118]:
forest = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=rng)
gradient_boost = XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.2, random_state=rng)


forest = forest.fit(X_train, y_train_writing)
forest_writing = forest.predict(X_test)

boost = gradient_boost.fit(X_train, y_train_writing)
boost_writing = boost.predict(X_test)

In [126]:
writing_forest_metrics = regression_metrics(y_test_writing, forest_writing)
writing_boost_metrics = regression_metrics(y_test_writing, boost_writing)

print(f'Writing score random forest regression metrics:\n{writing_forest_metrics}')
print(f'\nWriting score XGBoost metrics:\n{writing_boost_metrics}')

Writing score random forest regression metrics:
{'MAE': 10.145022910450555, 'MSE': 167.31135117802864, 'RMSE': 12.934888912473452, 'R2': 0.16666398364606272}

Writing score XGBoost metrics:
{'MAE': 10.295299396514892, 'MSE': 168.79804049898644, 'RMSE': 12.992230004852379, 'R2': 0.1592591557753904}


# Summary

Given the nature of what we are aiming to predict, the mean absolute error (MAE) is a simple and accurate metric to use as a reference between models

In [141]:
metrics = {'Subject':['Math', 'Reading', 'Writing'], 
           'Random Forrest MAE':[math_forest_metrics['MAE'], reading_forest_metrics['MAE'], writing_forest_metrics['MAE']],
           'XGBoost MAE':[math_boost_metrics['MAE'], reading_boost_metrics['MAE'], writing_boost_metrics['MAE']]
           }
metrics_df = pd.DataFrame(metrics)
metrics_df

,Subject,Random Forrest MAE,XGBoost MAE
0,Math,11.293742,11.288141
1,Reading,11.404424,11.745195
2,Writing,10.145023,10.295299


Put into the context of predicting the exam results of a student given the data collected on them about GENDER, RACE, PARENT_EDUCATION, LUNCH and TEST_PREP, we can predict their **math and reading scores with about 11% uncertainty** about **10% for that or writing**.